In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from tqdm import tqdm
base_path = '/Users/freezylex/Documents/course_work/'

In [2]:
svensson_params = pd.read_csv(os.path.join(base_path, 'data/raw_data/svenson_cbr_params.csv'), 
                             header=0, sep = ';', 
                             parse_dates = True, 
                             decimal=',', ).dropna(axis='rows')
svensson_params['Date'] = svensson_params['Date'].apply(lambda x: x.replace('.', '-'))
svensson_params['Date'] = pd.to_datetime(svensson_params['Date'], format = '%d-%m-%Y')
svensson_params.set_index('Date', inplace=True)
svensson_params.head(2)

,B1,B2,B3,T1,G1,G2,G3,G4,G5,G6,G7,G8,G9
Date,,,,,,,,,,,,,
2015-03-02,804.725297,468.880095,843.447322,3.736370,-8.560755,-12.082178,-10.583440,-6.865038,53.983070,12.999599,-8.849073,0.0,0.0
2015-03-03,782.256644,421.861743,863.867577,3.770833,-50.607897,-20.677139,12.823329,9.897471,17.886957,5.188354,3.799115,0.0,0.0


In [3]:
def g_curve_on_date(input_data, t):
    '''
    Function for Svensson curve values.
    
    input:
    - input_data: slice of svensson_params DataFrame 
      on a particular day, pandas.core.frame.DataFrame
    - t: duration of bond, float
    
    return:
    - G-curve value for a specific duration, float
    '''
    k = 1.6
    a = [0, 0.6, 1.56, 3.096, 5.5536, 9.48576, 15.777216, 25.8435456, 41.94967296]
    b = [0.6, 0.96, 1.536, 2.4576, 3.93216, 6.291456, 10.0663296, 16.10612736, 25.769803776]
    part_1 = input_data['B1'] + (input_data['B2'] + input_data['B3'])*\
            (input_data['T1'] / t)*(1 - np.exp(-t/input_data['T1']))
    part_2 = np.exp(-t/input_data['T1']) * (-1) * input_data['B3']
    part_3 = 0
    for i in range(1, 10):
        part_3 += input_data['G' + str(i)]*np.exp((((t - a[i-1]) ** 2) / (b[i-1] ** 2))*(-1))
    result = part_1 + part_2 + part_3
    return (np.exp(result / 10 ** 4) - 1)

In [4]:
def return_time_until_red(curr_date, redemption_date):
    date_red = pd.to_datetime(redemption_date, format = '%d.%m.%Y').date()
    return (date_red - curr_date.date()).days / 365

In [5]:
def return_bonds_theor_yield(bonds_file, svensson):
    columns_for_df = list(bonds_file.keys())
    index_for_df = svensson.index
    target_df = pd.DataFrame(columns = columns_for_df,
                             index = index_for_df)
    for bond in columns_for_df:
        yields = []
        for date in index_for_df:
            til_red = return_time_until_red(date, bonds_file[bond]['redemption_date'])
            bond_yield = g_curve_on_date(svensson.loc[date], til_red)
            yields.append(bond_yield)
        target_df[bond] = yields
    return target_df

In [6]:
import json
file = open(os.path.join(base_path, 'data/raw_data/bonds_info.json'), 'r')
data_bonds = json.load(file)
final = return_bonds_theor_yield(data_bonds, svensson_params)

In [7]:
final

,SU26214RMFS,SU26215RMFS,SU26216RMFS,SU26217RMFS,SU26218RMFS,SU26203RMFS,SU26204RMFS,SU26205RMFS,SU26206RMFS,SU26207RMFS,SU26208RMFS,SU26209RMFS,SU26210RMFS,SU26211RMFS,SU26212RMFS
Date,,,,,,,,,,,,,,,
2015-03-02,0.145507,0.135984,0.146208,0.142918,0.114296,0.140930,0.145273,0.143824,0.143718,0.124295,0.146177,0.139973,0.145992,0.138089,0.121807
2015-03-03,0.137546,0.129825,0.139080,0.134939,0.112495,0.136228,0.139801,0.135735,0.139331,0.121197,0.139295,0.132655,0.138320,0.131303,0.119147
2015-03-04,0.138207,0.131216,0.139465,0.135858,0.115474,0.135103,0.139427,0.136580,0.138220,0.123399,0.139594,0.133784,0.138878,0.132556,0.121548
2015-03-05,0.135256,0.129623,0.136374,0.133318,0.115313,0.133051,0.136558,0.133904,0.135702,0.122948,0.136509,0.131661,0.135835,0.130688,0.121214
2015-03-06,0.134577,0.129742,0.135296,0.133021,0.115847,0.130643,0.135021,0.133509,0.133881,0.123398,0.135343,0.131589,0.134986,0.130718,0.121715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-02-15,0.103115,0.102251,0.103022,0.102981,0.100313,0.093857,0.102539,0.103045,0.101230,0.100789,0.102976,0.102712,0.103098,0.102505,0.100584
2016-02-16,0.103276,0.102734,0.102823,0.103282,0.100992,0.094776,0.101761,0.103316,0.100848,0.101505,0.102659,0.103093,0.103145,0.102935,0.101318
2016-02-17,0.102453,0.101744,0.102113,0.102349,0.100269,0.096522,0.101233,0.102411,0.100740,0.100618,0.101975,0.102113,0.102369,0.101945,0.100464


In [8]:
final.to_excel(os.path.join(base_path, 'data/theor_target.xlsx'))

## Create the curve for further application

In [9]:
periods = [1/12, 2/12, 3/12, 6/12, 9/12, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20]
curve_values = pd.DataFrame(index = svensson_params.index, columns = periods)
for val in tqdm(curve_values.index):
    for period in periods:
        curve_values.loc[val, period] = g_curve_on_date(svensson_params.loc[val], period)

100%|██████████| 245/245 [00:01<00:00, 160.71it/s]


## curve_values

In [11]:
curve_values.to_csv(os.path.join(base_path, 'data/metrics/for_PCA.csv'))